# Module 3: Networking & Consensus

## Overview
This notebook explores networking and consensus mechanisms in blockchain systems. By the end of this module, you will understand:

1. Peer-to-peer network implementation
2. Node discovery protocols
3. Message propagation
4. Consensus algorithms

## Table of Contents
1. [P2P Networking](#p2p)
2. [Node Discovery](#discovery)
3. [Message Propagation](#propagation)
4. [Consensus Algorithms](#consensus)
5. [Implementation Examples](#implementation)
6. [Summary](#summary)

<a id='p2p'></a>
## 1. P2P Networking

### What is P2P Networking?
Peer-to-peer (P2P) networking is a distributed application architecture that partitions tasks or workloads between peers. In blockchain systems, P2P networking enables:

1. **Decentralization**: No central authority or single point of failure
2. **Scalability**: Network capacity increases with the number of nodes
3. **Fault Tolerance**: Network continues to operate even if some nodes fail

### P2P Network Characteristics
1. **Unstructured**: Nodes connect randomly
2. **Structured**: Nodes organized in a specific topology (e.g., DHT)
3. **Hybrid**: Combination of structured and unstructured approaches

### Node Structure
```go
// Node represents a P2P node in the network
type Node struct {
    Address    string
    Port       int
    Peers      map[string]*Peer
    peersMutex sync.RWMutex
    Server     *http.Server
    Blockchain *Blockchain
}

// Peer represents a peer in the network
type Peer struct {
    Address string
    Port    int
    Conn    net.Conn
}
```

<a id='discovery'></a>
## 2. Node Discovery

### What is Node Discovery?
Node discovery is the process by which nodes in a P2P network find and connect to each other. This is essential for network formation and maintenance.

### Discovery Methods
1. **Bootstrap Nodes**: Pre-configured nodes that help new nodes join the network
2. **Peer Exchange**: Nodes share their peer lists with each other
3. **Broadcast Discovery**: Nodes broadcast their presence on the network

### Bootstrap Process
```go
// DiscoverPeers discovers new peers in the network
func (n *Node) DiscoverPeers(bootstrapNodes []string) {
    fmt.Println("Discovering peers...")

    // In a real implementation, this would:
    // 1. Connect to bootstrap nodes
    // 2. Request peer lists
    // 3. Add discovered peers to the peer list
    // 4. Recursively discover more peers

    for _, bootstrapNode := range bootstrapNodes {
        fmt.Printf("Connecting to bootstrap node: %s\n", bootstrapNode)
        // Add logic to connect and discover peers
    }
}
```

### Adding Peers
```go
// AddPeer adds a peer to the node
func (n *Node) AddPeer(address string, port int) error {
    n.peersMutex.Lock()
    defer n.peersMutex.Unlock()

    peerAddress := fmt.Sprintf("%s:%d", address, port)

    // Check if peer already exists
    if _, exists := n.Peers[peerAddress]; exists {
        return fmt.Errorf("peer already exists")
    }

    // Connect to peer
    conn, err := net.Dial("tcp", peerAddress)
    if err != nil {
        return err
    }

    // Create peer
    peer := &Peer{
        Address: address,
        Port:    port,
        Conn:    conn,
    }

    // Add peer to map
    n.Peers[peerAddress] = peer

    fmt.Printf("Added peer: %s\n", peerAddress)
    return nil
}
```

<a id='propagation'></a>
## 3. Message Propagation

### What is Message Propagation?
Message propagation is the process of distributing information (blocks, transactions, etc.) throughout the network.

### Propagation Methods
1. **Flooding**: Send to all connected peers
2. **Gossip**: Randomly select peers to send messages to
3. **Tree-based**: Use a tree structure for efficient distribution

### Message Structure
```go
// Message represents a network message
type Message struct {
    Type    string
    Payload []byte
}

// HandleMessage handles an incoming message
func (n *Node) HandleMessage(msg *Message) {
    switch msg.Type {
    case "block":
        // Handle block message
        fmt.Println("Received block message")
        // In a real implementation, this would deserialize and add the block to the blockchain
    case "transaction":
        // Handle transaction message
        fmt.Println("Received transaction message")
        // In a real implementation, this would deserialize and add the transaction to the mempool
    case "get_blocks":
        // Handle get blocks request
        fmt.Println("Received get blocks request")
        // In a real implementation, this would send blocks to the requesting peer
    case "ping":
        // Handle ping message
        fmt.Println("Received ping message")
    default:
        fmt.Printf("Unknown message type: %s\n", msg.Type)
    }
}
```

### Broadcasting Messages
```go
// BroadcastMessage broadcasts a message to all peers
func (n *Node) BroadcastMessage(msg *Message) error {
    n.peersMutex.RLock()
    defer n.peersMutex.RUnlock()

    // Serialize message
    data, err := n.serializeMessage(msg)
    if err != nil {
        return err
    }

    // Send message to all peers
    for _, peer := range n.Peers {
        if peer.Conn != nil {
            _, err := peer.Conn.Write(data)
            if err != nil {
                fmt.Printf("Error sending message to peer %s:%d: %s\n", peer.Address, peer.Port, err)
            }
        }
    }

    return nil
}
```

<a id='consensus'></a>
## 4. Consensus Algorithms

### What is Consensus?
Consensus is the process by which nodes in a distributed network agree on the state of the system. In blockchain, this means agreeing on which transactions are valid and in what order they should be recorded.

### Types of Consensus Algorithms

#### 1. Proof of Work (PoW)
Nodes (miners) compete to solve a computationally difficult puzzle. The first to solve it gets to add the next block.

```go
// ProofOfWork represents the proof of work structure
type ProofOfWork struct {
    block  *Block
    target *big.Int
}

// Run performs the proof of work algorithm
func (pow *ProofOfWork) Run() (int, []byte) {
    var hashInt big.Int
    var hash [32]byte
    var nonce int = 0

    fmt.Printf("Mining a new block...")
    for nonce < math.MaxInt64 {
        data := pow.prepareData(nonce)
        hash = sha256.Sum256(data)
        fmt.Printf("\r%x", hash)
        hashInt.SetBytes(hash[:])

        if hashInt.Cmp(pow.target) == -1 {
            break
        } else {
            nonce++
        }
    }
    fmt.Print("\n\n")

    return nonce, hash[:]
}
```

#### 2. Proof of Stake (PoS)
Nodes are chosen to create new blocks based on the amount of cryptocurrency they hold and are willing to "stake" as collateral.

#### 3. Practical Byzantine Fault Tolerance (pBFT)
Nodes communicate with each other to reach consensus, tolerating up to ⌊(n-1)/3⌋ Byzantine failures where n is the total number of nodes.

#### 4. Delegated Proof of Stake (DPoS)
Token holders vote for delegates who are responsible for validating transactions and creating blocks.

<a id='implementation'></a>
## 5. Implementation Examples

### Starting a Node
```go
// Start starts the P2P node
func (n *Node) Start() error {
    fmt.Printf("Starting node at %s:%d\n", n.Address, n.Port)

    // Start listening for connections
    listener, err := net.Listen("tcp", fmt.Sprintf("%s:%d", n.Address, n.Port))
    if err != nil {
        return err
    }
    defer listener.Close()

    // Start HTTP server in a goroutine
    go func() {
        if err := n.Server.ListenAndServe(); err != nil && err != http.ErrServerClosed {
            fmt.Printf("HTTP server error: %s\n", err)
        }
    }()

    // Accept incoming connections
    for {
        conn, err := listener.Accept()
        if err != nil {
            fmt.Printf("Error accepting connection: %s\n", err)
            continue
        }

        // Handle connection in a goroutine
        go n.handleConnection(conn)
    }
}
```

### Handling Connections
```go
// handleConnection handles an incoming connection
func (n *Node) handleConnection(conn net.Conn) {
    defer conn.Close()

    // Read data from connection
    buffer := make([]byte, 4096)
    for {
        nBytes, err := conn.Read(buffer)
        if err != nil {
            fmt.Printf("Error reading from connection: %s\n", err)
            return
        }

        // Deserialize message
        msg, err := n.deserializeMessage(buffer[:nBytes])
        if err != nil {
            fmt.Printf("Error deserializing message: %s\n", err)
            continue
        }

        // Handle message
        n.HandleMessage(msg)
    }
}
```

<a id='summary'></a>
## 6. Summary

In this module, we've covered networking and consensus mechanisms:

1. **P2P Networking**: The foundation of decentralized blockchain networks
2. **Node Discovery**: How nodes find and connect to each other
3. **Message Propagation**: Distributing information throughout the network
4. **Consensus Algorithms**: Ensuring agreement on the blockchain state

### Next Steps
In Module 4, we'll explore enterprise blockchain features:
- Permissioned blockchains
- Hyperledger Fabric
- Scalability solutions

### Key Takeaways
- P2P networking enables decentralization and fault tolerance
- Node discovery is crucial for network formation
- Message propagation ensures information reaches all nodes
- Consensus algorithms maintain network integrity and agreement

### Practice Exercises
1. Implement a simple P2P node that can connect to peers
2. Create a message broadcasting system
3. Build a basic Proof of Work consensus mechanism
4. Design a node discovery protocol using bootstrap nodes